In [1]:
import os
from huggingface_hub import login
accesskey = os.getenv("HF_ACCESSKEY") 
login(accesskey)

In [9]:
import torch, gc

del model  # hvis lastet tidligere
gc.collect()
torch.cuda.empty_cache()


In [1]:
# 02_run_hf.ipynb

from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import pandas as pd
import torch
import time

# Load test data
file_path = "../data/merged_test_set.xlsx"
df = pd.read_excel(file_path)

# Load HF model and tokenizer
model_name = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)



PROMPT_TEMPLATE = (
    'The following sentence is in {language}. '
    'Rate its grammatical acceptability on a scale from 1 (completely unacceptable) to 5 (fully acceptable). '
    'Just answer with a number.\n\n"{sentence}"'
)


import re

def query_model(sentence, lang, model, tokenizer, device="cuda:0"):
    prompt = PROMPT_TEMPLATE.format(language=lang, sentence=sentence)
    inputs = tokenizer(prompt, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}


    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=10,
            temperature=0.0,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    reply = decoded.replace(prompt, "").strip().split()[0]
    return reply

# Run evaluation (limit to N examples)
results = []
N = 10  # increase when ready

import time
import pandas as pd

results = []
for i, row in df.iloc[:N].iterrows():
    sent = row["sentence"]
    lang = "Norwegian" if row["lang"] == "no" else "English"
    score = query_model(sent, lang, model, tokenizer)
    print(f"{i}: {sent} ({lang}) -> {score}")
    results.append(score)
    time.sleep(0.5)

# Lagre resultatene
out_df = df.iloc[:N].copy()
out_df["hf_mistral_score"] = results
out_df.to_csv("../results/scores_hf_mistral.csv", index=False)
print("Done.")


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


0: Hvilken artikkel kastet du uten å lese (Norwegian) -> 1.


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


1: Hvilken artikkel kastet du uten å lese den (Norwegian) -> I'd


...

7: Which article did you throw it away without reading it (English) -> 1.


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


8: Hvilket rom ryddet han før han malte (Norwegian) -> I'd


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


9: Hvilket rom ryddet han før han malte det (Norwegian) -> I'd
Done.
